In [1]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", None)

In [2]:
DATA_DIR = Path("../data")

In [3]:
ingredients = pd.read_csv(DATA_DIR / "ingredients.csv")
recipes = pd.read_csv(DATA_DIR / "recipes.csv")
recipe_ingredients = pd.read_csv(DATA_DIR / "recipe_ingredients.csv")
pantry = pd.read_csv(DATA_DIR / "pantry.csv")

In [4]:
# If the same ingredient exists in multiple locations, sum it.
pantry_agg = (
    pantry
    .groupby("ingredient_id", as_index=False)
    .agg({"quantity": "sum"})
)

display(pantry_agg)

,ingredient_id,quantity
0,1,3
1,3,1000
2,5,500
3,6,500
4,9,250


In [5]:
# Join Pantry With Recipe Ingredients

recipe_ing = recipe_ingredients.merge(
    pantry_agg,
    on="ingredient_id",
    how="left"
)

recipe_ing = recipe_ing.rename(columns={"quantity_y": "available_quantity"})
recipe_ing["available_quantity"] = recipe_ing["available_quantity"].fillna(0)
recipe_ing = recipe_ing.rename(columns={"quantity_x": "required_quantity"})


display(recipe_ing.head())

,recipe_id,ingredient_id,required_quantity,unit,is_optional,available_quantity
0,1,1,1.0,pieces,False,3.0
1,1,2,1.0,pieces,False,0.0
2,1,3,100.0,grams,False,1000.0
3,1,4,0.5,tsp,False,0.0
4,1,8,0.5,tsp,False,0.0


In [6]:
# Determine Ingredient Availability Per Recipe

recipe_ing["is_available"] = (
    recipe_ing["available_quantity"] >= recipe_ing["required_quantity"]
)

recipe_ing.loc[recipe_ing["is_optional"], "is_available"] = True

In [7]:
def ingredient_status(row):
    if row["is_optional"]:
        return "optional"
    elif row["is_available"]:
        return "available"
    elif row["available_quantity"] > 0:
        return "partial"
    else:
        return "missing"

recipe_ing["status"] = recipe_ing.apply(ingredient_status, axis=1)

display(
    recipe_ing[
        ["recipe_id", "ingredient_id", "required_quantity",
         "available_quantity", "status"]
    ].head(10)
)

,recipe_id,ingredient_id,required_quantity,available_quantity,status
0,1,1,1.0,3.0,available
1,1,2,1.0,0.0,missing
2,1,3,100.0,1000.0,available
3,1,4,0.5,0.0,missing
4,1,8,0.5,0.0,missing
5,2,6,150.0,500.0,available
6,2,1,0.5,3.0,optional
7,2,7,1.0,0.0,optional
8,4,9,50.0,250.0,available
9,4,5,100.0,500.0,available


In [8]:
# recipe level metrics
recipe_metrics = (
    recipe_ing[~recipe_ing["is_optional"]]
    .groupby("recipe_id")
    .agg(
        total_ingredients=("ingredient_id", "count"),
        available_count=("is_available", "sum"),
        missing_count=("status", lambda x: (x == "missing").sum()),
        partial_count=("status", lambda x: (x == "partial").sum())
    )
    .reset_index()
)

recipe_metrics["pantry_match_pct"] = (
    recipe_metrics["available_count"] / recipe_metrics["total_ingredients"]
) * 100

display(recipe_metrics)

,recipe_id,total_ingredients,available_count,missing_count,partial_count,pantry_match_pct
0,1,5,2,3,0,40.0
1,2,1,1,0,0,100.0
2,4,2,2,0,0,100.0


In [9]:
# Attach Recipe Metadata
recipe_scores = recipe_metrics.merge(
    recipes,
    on="recipe_id",
    how="left"
)

display(
    recipe_scores[
        ["recipe_id", "name", "dish_type", "pantry_match_pct",
         "missing_count", "partial_count"]
    ]
)

,recipe_id,name,dish_type,pantry_match_pct,missing_count,partial_count
0,1,Vegetable Upma,breakfast,40.0,3,0
1,2,Chana Salad,salad,100.0,0,0
2,4,Overnight Oats,breakfast,100.0,0,0


In [10]:
missing_ingredients = (
    recipe_ing[recipe_ing["status"] == "missing"]
    .merge(ingredients, on="ingredient_id", how="left")
    .groupby("recipe_id")["name"]
    .apply(list)
    .reset_index(name="missing_ingredients")
)

recipe_scores = recipe_scores.merge(
    missing_ingredients,
    on="recipe_id",
    how="left"
)

recipe_scores["missing_ingredients"] = recipe_scores["missing_ingredients"].fillna('[]')

display(
    recipe_scores[
        ["name", "pantry_match_pct", "missing_ingredients"]
    ]
)


,name,pantry_match_pct,missing_ingredients
0,Vegetable Upma,40.0,"[tomato, cumin seeds, salt]"
1,Chana Salad,100.0,[]
2,Overnight Oats,100.0,[]


In [ ]:
# Sort Recipes by Pantry Match

ranked_recipes = recipe_scores.sort_values(
    by=["pantry_match_pct", "missing_count"],
    ascending=[False, True]
)

display(
    ranked_recipes[
        ["name", "pantry_match_pct", "missing_count", "missing_ingredients"]
    ]
)


,name,pantry_match_pct,missing_count,missing_ingredients
1,Chana Salad,100.0,0,[]
2,Overnight Oats,100.0,0,[]
0,Vegetable Upma,40.0,3,"[tomato, cumin seeds, salt]"


In [ ]:
# One-Recipe Deep Dive (Debugging Tool)

def inspect_recipe(recipe_name):
    recipe_id = recipes.loc[
        recipes["name"] == recipe_name, "recipe_id"
    ].iloc[0]
    
    display(
        recipe_ing[
            recipe_ing["recipe_id"] == recipe_id
        ].merge(ingredients, on="ingredient_id")
        [["name", "required_quantity", "available_quantity", "status"]]
    )

inspect_recipe("Vegetable Upma")

,name,required_quantity,available_quantity,status
0,onion,1.0,3.0,available
1,tomato,1.0,0.0,missing
2,rice,100.0,1000.0,available
3,cumin seeds,0.5,0.0,missing
4,salt,0.5,0.0,missing
